In [9]:
# From https://stackabuse.com/text-generation-with-python-and-tensorflow-keras/

import pandas as pd
import pickle
import numpy
import sys
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

# nltk.download('stopwords')

titles = pd.DataFrame(pickle.load(open("pickle/features.p", "rb" )))['title'].tolist()

In [12]:
titles[-10:]

['Danza (CT 085)',
 'Danza (CT 087)',
 'Danza (CT 087)',
 'Danza (CT 103)',
 'Danza (CT 103)',
 'Danza (CT 129)',
 'Danza (CT 129)',
 'Danza (CT 159)',
 'Danza (CT 159)',
 'Danza (CT 161)']

In [13]:
def tokenize_words(input):
    # lowercase everything to standardize it
    input = input.lower()

    # instantiate the tokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(input)

    # if the created token isn't in the stop words, make it part of "filtered"
    # filtered = filter(lambda token: token not in stopwords.words('english'), tokens)
    return " ".join(filtered)

In [14]:
# preprocess the input data, make tokens
processed_inputs = tokenize_words(titles)
processed_inputs

AttributeError: 'list' object has no attribute 'lower'

In [21]:
chars = sorted(list(set(processed_inputs)))
char_to_num = dict((c, i) for i, c in enumerate(chars))

In [22]:
input_len = len(processed_inputs)
vocab_len = len(chars)
print ("Total number of characters:", input_len)
print ("Total vocab:", vocab_len)

Total number of characters: 99550
Total vocab: 41


In [23]:
seq_length = 100
x_data = []
y_data = []

In [24]:
# loop through inputs, start at the beginning and go until we hit
# the final character we can create a sequence out of
for i in range(0, input_len - seq_length, 1):
    # Define input and output sequences
    # Input is the current character plus desired sequence length
    in_seq = processed_inputs[i:i + seq_length]

    # Out sequence is the initial character plus total sequence length
    out_seq = processed_inputs[i + seq_length]

    # We now convert list of characters to integers based on
    # previously and add the values to our lists
    x_data.append([char_to_num[char] for char in in_seq])
    y_data.append(char_to_num[out_seq])

In [25]:
n_patterns = len(x_data)
print ("Total Patterns:", n_patterns)

Total Patterns: 99450


In [26]:
X = numpy.reshape(x_data, (n_patterns, seq_length, 1))
X = X/float(vocab_len)

In [27]:
y = np_utils.to_categorical(y_data)

In [28]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [29]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [30]:
filepath = "model_weights_saved.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]

In [31]:
model.fit(X, y, epochs=4, batch_size=256, callbacks=desired_callbacks)

Instructions for updating:
Use tf.cast instead.
Epoch 1/4
 5376/99450 [>.............................] - ETA: 49:47 - loss: 3.1867

KeyboardInterrupt: 

In [ ]:
filename = "model_weights_saved.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
num_to_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
start = numpy.random.randint(0, len(x_data) - 1)
pattern = x_data[start]
print("Random Seed:")
print("\"", ''.join([num_to_char[value] for value in pattern]), "\"")